In [239]:
import re

# Specify the path to your text file
file_path = '34_min.txt'

# Read the contents of the file
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()

# Replace all occurrences of the pattern '(hh:mm) ' with a newline
modified_content = re.sub(r'\(\d{2}:\d{2}\) ', '\n', content)

# Write the modified content back to the same file
with open(file_path, 'w', encoding='utf-8') as file:
    file.write(modified_content)

print("The file has been successfully updated.")


The file has been successfully updated.


In [240]:
import warnings
warnings.filterwarnings("ignore")
# warnings.simplefilter('ignore')

from langchain_community.document_loaders import Docx2txtLoader, TextLoader
from langchain_community.embeddings import AzureOpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationChain, RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.chat_models.azure_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
import os


In [241]:
os.environ["OPENAI_API_KEY"] = "b7d5aa82d15a4b99a1c730f681ec2bbc"
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["OPENAI_API_BASE"] = "https://hmh-digitalhub-azure-openai.openai.azure.com/"
os.environ["CHAT_MODEL"] = "gpt-35-turbo"
os.environ["CHAT_MODEL_DEPLOYMENT_NAME"] = "gpt-35-turbo"
os.environ["EMBEDDINGS_MODEL"] = "text-embedding-ada-002"
os.environ["EMBEDDINGS_MODEL_DEPLOYMENT_NAME"] = "text-embedding-ada-002"

llm = AzureChatOpenAI(  
            model_name ="gpt-35-turbo",
            deployment_name= "gpt-35-turbo",
            temperature=0,
            openai_api_version ="2023-07-01-preview",
            openai_api_key="b7d5aa82d15a4b99a1c730f681ec2bbc",
            openai_api_base="https://hmh-digitalhub-azure-openai.openai.azure.com/"
        )

embeddings = AzureOpenAIEmbeddings()

In [215]:
documents = []
loader = Docx2txtLoader("transcript/gen_ai.docx")    
documents.extend(loader.load())

In [242]:
documents = []
loader = TextLoader("34_min.txt")    
documents.extend(loader.load())

In [243]:
len(documents[0].page_content)

39740

In [244]:
max_para_len= 0
for para in documents[0].page_content.split('\n\n'):
    max_para_len= max(max_para_len, len(para))

print(max_para_len)

339


In [245]:
text_splitter = CharacterTextSplitter(chunk_size= 1000, chunk_overlap= 0)
chunked_documents = text_splitter.split_documents(documents)
len(chunked_documents)

43

In [246]:
for doc in chunked_documents:
    print(len(doc.page_content))

919
963
963
954
959
952
953
937
960
957
946
961
990
956
970
959
918
945
961
942
919
946
971
948
970
953
671
974
968
970
671
667
950
939
948
943
926
971
949
955
977
938
466


In [247]:
db = FAISS.from_documents(chunked_documents, embeddings)

In [270]:
# query= 'Can I create good keyframes?'
# query= 'What is attension mechanism?'
# query= 'How is the audiobook getting created?'
# query= 'Explain step by step how the feature film is getting created from the ebook.'
# 12_min.txt
# query= 'What are the key points for the better working of the model?'
# query= 'How can I learn about different chain types?'
# query= 'Why are we using custom prompts?'
# query= 'What is the flow of the application from starting to end?'
# query= 'Why do we not want our user to do prompt engineering to get the result in the format he wants? And are we helping the user to get the output in the format he wants?'
# query= 'Who is selling the product and to whom? Also give the name of the product.'
# query= 'How are we creating embeddings of the text?'
# query= 'Explain step by step how are we setting up the chat prompt template? How are we taking the user input into it?'
query= 'Who is Mike noop?'
# query= ''

In [257]:

documents= db.similarity_search_with_score(query)

In [258]:
for doc in documents:
    print(doc[1])
    print(doc[0].page_content)
    print('-----------------------------------------------------------------------------------------')

0.44228512
we we sort of call the code right moment internally That's Mike noop co-founder and head of AI at zapier he sled the product team that helps over 10 million people automate workflows between 5 000 of your favorite applications many consider zapier to be the backbone of modern Automation in early 2022 Mike

gave up his exact title to go all in on AI and zapier not just for product's sake but to figure out how to enable zapier's internal employees with AI I wanted to talk with Mike because he thinks about how to enable his employees with AI in ways that other Business Leaders don't gotten from the whole work

and just said yeah we're gonna give everybody one week to go like step out of your day job and figure out how to use language models for yourself in this episode we talk about zapier's top internal use cases for AI that actually increase Revenue Mike's recommendations for how other businesses should get
---------------------------------------------------------------------

In [259]:
retriever= db.as_retriever(search_kwargs={"k": 6})

In [260]:
prompt= ChatPromptTemplate.from_template('''
Answer the following question based only on the provided context. 
Do not respond with anything outside of the context. If you don't know, say, "I don't know"
Think step by step before providing me detailed answer.
<context>
{context}
</context>
Question: {input}
''')

In [278]:
document_chain= create_stuff_documents_chain(llm, prompt)
retrieval_chain= create_retrieval_chain(retriever, document_chain)

In [273]:
response= retrieval_chain.invoke({'input': query})

TypeError: RunnableParallel.invoke() got an unexpected keyword argument 'verbose'

In [272]:
print(response['answer'])

I don't know
